## Knapsack Problem

The *knapsack problem*:  given a set of integers $S = \{s_1, s_2, \ldots, s_n\}$ and a target number $T$, find a subset (i.e., knapsack) of $S$ which adds up exactly to $T$.  

For example, if $S = \{1,2,5, 9, 10\}$, there is a subset that adds up to $T = 22$, but not to $T = 23$.  Complete the following tasks related to this problem.

# 1. 

Find a subset of $S = \{1,2,5, 9, 10\}$ with sum $T = 22$.  Explain the process (algorithm) you used mentally to find the subset.  Then apply the same process in an attempt to find a subset with sum $T = 23$.  

How do you know there is no such subset?

In [1]:
def subsets(S, T):
    n = len(S)
    """
    Start by using algorithm to list out all possible subsets of S
    """
    for i in range(1 << n):
        SS = [S[j] for j in range(n) if (i & (1 << j))]
        """
        Using the acquired subset, determine if the sum of the subset is equivalent to T
        """
        if sum(SS) == T:
            return SS

S = [1,2,5,9,10]
print(subsets(S, 22))
print(subsets(S, 23)) ## this isn't possible as the subset to attain 22 is {1, 2, 9, 10}, 5 can't be add/subtracted to help

[1, 2, 9, 10]
None


# 2.

Consider the following possible algorithm for the knapsack problem, written in psuedocode: 
```python
knapsack(S[], T):
    K = empty
    for each i < size(S)
        if sum(K) + S[i] <= T, put S[i] into K
    if sum(K) = T, return K, else return False.
```
**a)** Describe what this algorithm does in English.  

**b)** Implement this algorithm in Python and run it on the $S$ and $T$ above.


**c)** Prove that this algorithm is NOT correct.  That is, find a counterexample: a set $S$ and number $T$ for which there is a solution, but not one that the algorithm finds.

**d)** Verify that this particular $S$ and $T$ does not give the right output when entered to your Python program.

## 2a
take a placeholder K to represent your sum
run through indexes smaller than the size of S
one by one, add in your values from set S into your placeholder K up until your expected value T
repeat until you get to your value T otherwise, return false 
false means the S does not have a subset which can add up to value K

In [2]:
## 2b
def knapsack(S, T):
    K = []
    for i in range(len(S)):
        if sum(K) + S[i] <= T:
            K.append(S[i])
        if sum(K) == T:
            return K
        else:
            return False

knapsack([1,2,5,9,10], 22)

False

## 2c
Seems the algorthim does not consider the possibility of using alternative options 
the algorithm only considers sum and append of values in the original set in order
does not "skip" values or go out of order to determine the final possible sum T

In [3]:
## 2d
knapsack([1,2,5,9,10], 5)

False

# 3. 

Another try: What if you put the elements in the knapsack from largest to smallest?  Check that this too is not a correct algorithm.

In [4]:
knapsack([10,9,5,2,1], 5)

False

# 4.

Describe a correct algorithm for the knapsack problem (that we haven't seen in class), both in English and in pseudocode.  Then implement the algorithm in Python.  Explain how you know your algorithm is correct (even if it might not be efficient).

### Logic and Pseudocode
Use an algorithm to list out subsets of the weights and then using the same algorithm to match a subset of the values based on the indices in the weight. Then compare the sum of the weight subsets with the bag size and only take values which match the bag size. Append the subsets of the values as well as sums to two lists accordingly. Finally, take the index with the maximum sum and use it to retrieve the items requirement for as well as the maximum sum for the knapsack.

```python
brutal_knapsack(bag_size,weights,values,n_items):
    pot = empty
    sums = empty
    for each i < size(n_items)
        SS = weights[j] for j in size(n_items) if (i & (1 << j))
        KS = values[j] for j in size(n_items) if (i & (1 << j))
        if sum(SS) <= bag_size, put KS into pot and sum(KS) into sums
    print("items:",pot[np.argmax(sums)],"give a maximum value of",sum(pot[np.argmax(sums)]))
```
    
The algorithm is correct as it forcibly computes all possible subsets, even ones which will return a bad value. Then compares all possible results to the restrictions (bag_size) placed to remove all invalids. 

In [5]:
def brutal_knapsack(bag_size, weights, values, n_items):
    import numpy as np
    pot = []
    sums = []
    for i in range(1 << n_items):
        SS = [weights[j] for j in range(n_items) if (i & (1 << j))]
        KS = [values[j] for j in range(n_items) if (i & (1 << j))]
        if sum(SS) <= bag_size:
            pot.append(KS)
            sums.append(sum(KS))
    print("items:",pot[np.argmax(sums)],"give a maximum value of",sum(pot[np.argmax(sums)]))
    """
    a simple 'return np.max(sums)' would provide the solution for the maximum of the knapsack. 
    This print provides the output with the items which would give that maximum value.
    """

bag_size = 7
weights = [2, 3, 4, 5]
values = [3, 4, 5, 5]
n_items = len(weights)

brutal_knapsack(bag_size, weights, values, n_items)

items: [4, 5] give a maximum value of 9


# 5. Generating correct change

Now, we will be making change using the fewest coins. 

Suppose you are a programmer for a vending machine manufacturer. Your company wants to streamline effort by giving out the fewest possible coins in change for each transaction. Suppose a customer puts in a dollar bill and purchases an item for 37 cents. What is the smallest number of coins you can use to make change? The answer is six coins: two quarters, one dime, and three pennies. 

How did we arrive at the answer of six coins? We start with the largest coin in our arsenal (a quarter) and use as many of those as possible, then we go to the next lowest coin value and use as many of those as possible. This is the greedy algorithm for change-making.

**Question:** Write the greedy algorithm for change making.

The input is the amount of change to generate (in pennies) and a list of coin sizes (in pennies)

The output is the minimum number of coins to gener

```
# buys with 1 dollar for 37 pennies
# Second argument says we can give quarters, dimes, nickels and pennies
make_change(100 - 37, [25, 10, 5, 1])

# 2 quarters, one dime, and three pennies
output --> 6 # Output would be equivalent to the choices [2, 1, 0, 3]
```

In [6]:
def make_change(remain, coins):
    change = []
    coins_count = []
    if remain < 0:
        print("You still owe", abs(remain), "cents.")
    elif remain == 0:
        print("You gave exact change, yay!")
    else:
        for i in range(len(coins)):
            while remain >= coins[i]:
                remain -= coins[i]
                change.append(coins[i])
        print("Your change is", len(change),"coins:", change.count(coins[0]),"quarter(s),",
              change.count(coins[1]),"dime(s),",
              change.count(coins[2]),"nickel(s), and",
              change.count(coins[3]),"pennies")

make_change(100 - 37, [25, 10, 5, 1])
make_change(100 - 137, [25, 10, 5, 1])
make_change(100 - 100, [25, 10, 5, 1])

Your change is 6 coins: 2 quarter(s), 1 dime(s), 0 nickel(s), and 3 pennies
You still owe 37 cents.
You gave exact change, yay!


# 6 Recursive change

Write the greedy change making algorithm using recursion

In [7]:
def recursive_change(remain, coins):
    def recursive_coins(i, num):
        if num == 0:
            return 0
        elif i == -1 or num < 0:
            return float('inf')
        else:
            return min(recursive_coins(i-1, num), 1 + recursive_coins(i, num-coins[i]))
    return recursive_coins(len(coins)-1, remain)

recursive_change(100 - 37, [25, 10, 5, 1])

6

# 7 Dynamic Programming Change making

Write a solution to the change making problem using dynamic programming.

**Hint:** Start with making change for one cent and systematically work its way up to the amount of change we require. This guarantees us that at each step of the algorithm we already know the minimum number of coins needed to make change for any smaller amount. Keep a memoized table of results for each step working up to the amount of change you need to generate.

In [8]:
def dynamic_change(remain, coins):
    m, n = len(coins)+1, remain+1
    table = [[0] * n for x in range(m)]
    for j in range(1, n):
        table[0][j] = float('inf')
    for i in range(1, m):
        for j in range(1, n):
            aC = table[i][j - coins[i-1]] if j - coins[i-1] >= 0 else float('inf')
            table[i][j] = min(table[i-1][j], 1 + aC)
    return table[m-1][n-1]

dynamic_change(100 - 37, [25, 10, 5, 1])

6